In [4]:
import json
from tqdm import tqdm
import pandas as pd
from simpletransformers.classification import MultiLabelClassificationModel
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import torch
from sklearn.model_selection import train_test_split

import random

import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [5]:
import psycopg2

con = psycopg2.connect(
  database="dsa1", 
  user="dsa1", 
  password="HSEPAassword2022", 
  host="77.37.164.38", 
  port="5432"
)

In [6]:
cur = con.cursor()
cur.execute("SELECT * from MIEM_PROJECTS")
rows = cur.fetchall()
column_names=["ID","NUMBER","VACANCIES","NAME","TYPE","TYPEDESC","TYPEID","STATUSID", "STATUSDESC","DIRECTIONHEAD","HEAD","DATACREATED","FACULTYID"]
df = pd.DataFrame(rows, columns=column_names)
df.head()

,ID,NUMBER,VACANCIES,NAME,TYPE,TYPEDESC,TYPEID,STATUSID,STATUSDESC,DIRECTIONHEAD,HEAD,DATACREATED,FACULTYID
0,852,NaN,2,Приготовление квантовых состояний для систем к...,nir,НИР,1,1,Готов к работе,,Шаховой Роман,10.08.2021 16:25:36,None
1,853,NaN,2,Создание системы квантового распределения ключ...,nir,НИР,1,1,Готов к работе,,Дуплинский Александр,10.08.2021 16:29:38,None
2,869,NaN,5,Разработка функциональной модели IP-блока (Сло...,startup,Стартап,4,1,Готов к работе,,Нефедов Сергей Игоревич,26.08.2021 14:32:28,None
3,967,NaN,1,Течение жидкостей в криволинейных трубах и кан...,nir,НИР,1,1,Готов к работе,,Данилов Владимир Григорьевич,30.09.2021 18:45:31,None
4,1026,NaN,2,Научные исследования в группе «Ультрахолодные ...,nir,НИР,1,1,Готов к работе,,Турлапов Андрей,20.10.2021 13:00:06,None


In [7]:
df.shape

(441, 13)

In [8]:
papers = pd.DataFrame({
    'title': df['NAME'].tolist(),
    'categories': df['TYPE'].tolist()
})

papers['title'] = papers['title'].apply(lambda x: x.replace("\n",""))
papers['title'] = papers['title'].apply(lambda x: x.strip())
papers['text'] = papers['title']

papers['categories'] = papers['categories'].apply(lambda x: tuple(x.split()))

shortlisted_categories = papers['categories'].value_counts().reset_index(name="count").query("count > 0")["index"].tolist()
papers = papers[papers["categories"].isin(shortlisted_categories)].reset_index(drop=True)

papers = papers.sample(frac=1).reset_index(drop=True)

papers = papers.groupby('categories').head(250).reset_index(drop=True)

multi_label_encoder = MultiLabelBinarizer()
multi_label_encoder.fit(papers['categories'])
papers['categories_encoded'] = papers['categories'].apply(lambda x: multi_label_encoder.transform([x])[0])

papers = papers[["text", "categories", "categories_encoded"]]
papers.head()

,text,categories,categories_encoded
0,Многофункциональный сценарный контроллер освещ...,"(soft-hard,)","[0, 0, 1, 0]"
1,Разработка и реализация методов идентификации ...,"(soft,)","[0, 1, 0, 0]"
2,Виртуальный космос виртуальный спутник,"(soft,)","[0, 1, 0, 0]"
3,Элементы сверхпроводящей спинтроники для приме...,"(nir,)","[1, 0, 0, 0]"
4,Исследование электрических и механических свой...,"(nir,)","[1, 0, 0, 0]"


In [9]:
papers.shape

(441, 3)

In [10]:
train, test = train_test_split(papers, test_size=0.3, stratify=papers['categories'], random_state=42)

Модель:

In [11]:
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "max_seq_length": 512,
    "train_batch_size": 4,
    "num_train_epochs": 4,
}

model = MultiLabelClassificationModel('bert', 
                                      'bert-base-cased', 
                                      num_labels=len(shortlisted_categories), 
                                      args=model_args)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassification were not in

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

In [12]:
model.train_model(train[['text', 'categories_encoded']])

/opt/conda/lib/python3.7/site-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/308 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/simpletransformers/classification/classification_utils.py:263: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:207.)
  labels = torch.tensor(labels, dtype=torch.long)
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/77 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/77 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/77 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/77 [00:00<?, ?it/s]

(308, 0.38940830823856515)

In [13]:
result, model_outputs, wrong_predictions = model.eval_model(test[['text', 'categories_encoded']])
print(result)

/opt/conda/lib/python3.7/site-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/133 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/17 [00:00<?, ?it/s]

{'LRAP': 0.8358395989974937, 'eval_loss': 0.3732467311270097}


In [16]:
for _ in range(5):

    random_idx = random.randint(0, len(test)-1)
    text = test.iloc[random_idx]['text']
    true_categories = test.iloc[random_idx]['categories']

    predicted_categories_encoded, raw_outputs = model.predict([text])
    predicted_categories_encoded = np.array(predicted_categories_encoded)
    predicted_categories_encoded[0][np.argmax(raw_outputs[0])] = 1
    predicted_categories = multi_label_encoder.inverse_transform(predicted_categories_encoded)[0]

    print(f'True Categories:'.ljust(21,' '), f'{true_categories} - {true_categories[0]}\n')
    print(f'Predicted Categories: {predicted_categories} - {predicted_categories[0]}\n')
    print(f'Abstract: {text}\n\n')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

True Categories:      ('nir',) - nir

Predicted Categories: ('nir',) - nir

Abstract: Геометрическое глубокое обучение




  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

True Categories:      ('nir',) - nir

Predicted Categories: ('nir',) - nir

Abstract: Построение инвестиционных портфелей с заданным значением дисперсии




  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

True Categories:      ('soft',) - soft

Predicted Categories: ('soft',) - soft

Abstract: Социальная платформа для игровых приложений


